In [25]:
import os
import os.path as osp

import numpy as np

import open3d as o3d

import utils

In [26]:
pcd_path = "data/kinth/pointcloud/1720509720.141261339.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)
# pcd = pcd.voxel_down_sample(voxel_size=0.01)
pcd = pcd.remove_radius_outlier(nb_points=25, radius=0.2)[0] # (pcd, new indexed from old)

points = np.asarray(pcd.points)
points = points - points.mean(axis=0)
print("max, min:", points.max(axis=0), points.min(axis=0))
print(f"before voxelized: {len(points)}")
points, _, _, _ = utils.voxel_downsample(points, 0.02, use_avg=True)
print("max, min:", points.max(axis=0), points.min(axis=0))
print(f"after voxelized: {len(points)}")

max, min: [4.71142875 3.22041954 1.96897121] [-3.35557116 -4.00158058 -1.04102878]
before voxelized: 12234
max, min: [4.71142875 3.22041954 1.96897121] [-3.35557116 -4.00158058 -1.04102878]
after voxelized: 7677


# height filtration

In [27]:
# pivot = utils.get_average_pivot(points[:, 2])
# points = points[points[:, 2] < pivot * 1.75]

# covariance rad feature

In [28]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

search_radius = 2.5
feat_rad_list = None
feat_rad_list_dump_file = f"data/kinth/pickle/feat_rad_{search_radius:.2f}_{osp.basename(pcd_path).split('.')[0]}.pkl"

# if osp.exists(feat_rad_list_dump_file):
#     with open(feat_rad_list_dump_file, "rb") as f:
#         feat_rad_list = pickle.load(f)
# else:
    # 多进程处理子空间坐标几何特征计算
num_worker = 4
batch_size = (len(points) - 1) // num_worker + 1 # round up
feat_rad_list_con = [None] * num_worker

with ProcessPoolExecutor(max_workers=num_worker) as executor:
    future_dict = {
        executor.submit(
            utils.eigval_radius,
            points.copy(),
            (i * batch_size, min((i + 1) * batch_size, len(points))),
            search_radius
        ) : i for i in range(num_worker)
    }
    for future in as_completed(future_dict):
        feat_list, neighbor_num_record = future.result()
        feat_rad_list_con[future_dict[future]] = feat_list
        print(f"missing:{(np.array(neighbor_num_record) < 3).astype(np.int32).sum()}")
feat_rad_list = np.concatenate(feat_rad_list_con, axis=0)
    
    # if not osp.exists(osp.dirname(feat_rad_list_dump_file)):
    #     os.makedirs(osp.dirname(feat_rad_list_dump_file), exist_ok=True)
    # with open(feat_rad_list_dump_file, "wb") as f:
    #     pickle.dump(feat_rad_list, f)

eigval progress: 100%|████████████████████████████████████████| 1920/1920 [00:00<00:00, 3220.30it/s]


0
0
0


eigval progress: 100%|████████████████████████████████████████| 1920/1920 [00:00<00:00, 2997.18it/s]


0


In [29]:
print(feat_rad_list)
feat_rad_color = feat_rad_list
feat_rad_color = feat_rad_color / (feat_rad_color.sum(axis=1).reshape(-1, 1) + 1e-9)
feat_rad_color = (feat_rad_color * 255.0).astype(np.int32)
print(feat_rad_color)


[[0.56340054 0.61862606 0.83349234]
 [0.4146207  0.69369646 0.82069625]
 [0.68543196 0.59812686 0.87358356]
 ...
 [0.29126786 0.96562089 0.97563442]
 [0.29126786 0.96562089 0.97563442]
 [0.29126786 0.96562089 0.97563442]]
[[ 71  78 105]
 [ 54  91 108]
 [ 81  70 103]
 ...
 [ 33 110 111]
 [ 33 110 111]
 [ 33 110 111]]


In [30]:
utils.npy2ply(points, feat_rad_color, "data/output/sample_rad.ply")

In [31]:
# empty slot for staged run

# covariance vtl feature

In [35]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import pickle

search_radius = 2.0
feat_vtl_list = None
feat_vtl_list_dump_file = f"data/kinth/pickle/feat_vtl_{search_radius:.2f}_{osp.basename(pcd_path).split('.')[0]}.pkl"

# if osp.exists(feat_vtl_list_dump_file):
#     with open(feat_vtl_list_dump_file, "rb") as f:
#         feat_vtl_list = pickle.load(f)
# else:
    # 多进程处理子空间坐标几何特征计算
num_worker = 4
batch_size = (len(points) - 1) // num_worker + 1 # round up
feat_vtl_list_con = [None] * num_worker

with ProcessPoolExecutor(max_workers=num_worker) as executor:
    future_dict = {
        executor.submit(
            utils.eigval_vertic,
            points,
            (i * batch_size, min((i + 1) * batch_size, len(points))),
            search_radius
        ) : i for i in range(num_worker)
    }
    for future in as_completed(future_dict):
        feat_list, neighbor_num_record = future.result()
        print(f"missing: {(np.array(neighbor_num_record) < 3).astype(np.int32).sum()}")
        feat_vtl_list_con[future_dict[future]] = feat_list
feat_vtl_list = np.concatenate(feat_vtl_list_con, axis=0)
    
    # if not osp.exists(osp.dirname(feat_vtl_list_dump_file)):
    #     os.makedirs(osp.dirname(feat_vtl_list_dump_file), exist_ok=True)
    # with open(feat_vtl_list_dump_file, "wb") as f:
    #     pickle.dump(feat_vtl_list, f)

eigval progress:   0%|                                                     | 0/1917 [00:00<?, ?it/s]

eigval progress: 100%|████████████████████████████████████████| 1920/1920 [00:00<00:00, 3585.87it/s]


missing: 0
missing: 0
missing: 0
missing: 0


In [36]:
print(feat_vtl_list)
feat_vtl_color = feat_vtl_list
feat_vtl_color = feat_vtl_color / (feat_vtl_color.sum(axis=1).reshape(-1, 1) + 1e-9)
feat_vtl_color = (feat_vtl_color * 255.0).astype(np.int32)
print(feat_vtl_color)

[[0.80679283 0.06703797 0.58701895]
 [0.80685799 0.06581422 0.58706786]
 [0.80663231 0.0700644  0.5868861 ]
 ...
 [0.75689113 0.37981457 0.53184275]
 [0.75910988 0.3609294  0.54173994]
 [0.75641498 0.40427009 0.51420043]]
[[140  11 102]
 [140  11 102]
 [140  12 102]
 ...
 [115  58  81]
 [116  55  83]
 [115  61  78]]


In [37]:
utils.npy2ply(points, feat_vtl_color, "data/output/sample_vtl.ply")